# 05 - Branched RAG (Multi-Query Retrieval)

**Complexity:** ⭐⭐⭐

**Use Cases:** Multi-intent queries, cross-domain research, comprehensive topic exploration

**Key Feature:** Generates multiple sub-queries from user question, retrieves in parallel for better coverage.

**Example:**
```
Query: "Compare OpenAI and HuggingFace embeddings for cost and performance"

Generated sub-queries:
1. "OpenAI embeddings pricing and cost"
2. "HuggingFace embeddings performance benchmarks"
3. "Comparison of embedding providers"

→ Retrieves diverse documents covering all aspects
```

## 1. Setup

In [ ]:
import sys
sys.path.append('../..')

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from shared.config import OPENAI_VECTOR_STORE_PATH, DEFAULT_MODEL
from shared.utils import load_vector_store, print_section_header, format_docs
from shared.prompts import RAG_PROMPT_TEMPLATE, MULTI_QUERY_PROMPT
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print_section_header("Setup: Branched RAG")

embeddings = OpenAIEmbeddings()
vectorstore = load_vector_store(OPENAI_VECTOR_STORE_PATH, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
llm = ChatOpenAI(model=DEFAULT_MODEL, temperature=0)

print("✅ Setup complete!")

## 2. Multi-Query Retriever

Uses `MultiQueryRetriever` to generate alternative queries and retrieve documents.

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
import logging

print_section_header("Multi-Query Retriever")

# Enable logging to see generated queries
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Create MultiQueryRetriever
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=llm,
    prompt=MULTI_QUERY_PROMPT
)

print("✓ MultiQueryRetriever created")
print("  - Generates 3 alternative queries")
print("  - Retrieves documents for each")
print("  - Deduplicates results")

## 3. Test Multi-Query Retrieval

In [ ]:
from shared.utils import print_results

print_section_header("Multi-Query Test")

query = "How to optimize RAG system performance?"
print(f"Original Query: '{query}'\n")

# Retrieve with multi-query
docs = multi_query_retriever.invoke(query)

print(f"\n✓ Retrieved {len(docs)} unique documents")
print_results(docs, "Multi-Query Results", max_docs=4, preview_length=150)

## 4. Build Branched RAG Chain

In [ ]:
print_section_header("Branched RAG Chain")

branched_chain = (
    {"context": multi_query_retriever | format_docs, "input": RunnablePassthrough()}
    | RAG_PROMPT_TEMPLATE
    | llm
    | StrOutputParser()
)

print("✓ Branched RAG chain created")

# Test
query = "What are the differences between similarity and MMR retrieval?"
print(f"\nQuery: '{query}'\n")
print("=" * 80)

response = branched_chain.invoke(query)
print(response)

print("\n" + "=" * 80)
print("\n✅ Branched RAG provides more comprehensive answers!")

## 5. Comparison: Simple vs Branched

In [ ]:
print_section_header("Comparison")

# Simple RAG
simple_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | RAG_PROMPT_TEMPLATE
    | llm
    | StrOutputParser()
)

query = "How to implement embeddings in RAG?"

print(f"Query: '{query}'\n")
print("[SIMPLE RAG]")
simple_docs = retriever.invoke(query)
print(f"Documents retrieved: {len(simple_docs)}")

print("\n[BRANCHED RAG]")
branched_docs = multi_query_retriever.invoke(query)
print(f"Documents retrieved: {len(branched_docs)} (from multiple queries)")

print("\n💡 Branched RAG typically retrieves more diverse documents")

## Summary

**Flow:**
```
User Query → Generate Sub-Queries → Parallel Retrieval → Deduplicate → LLM → Response
```

**Advantages:**
✅ Better coverage for complex queries  
✅ Captures multiple aspects  
✅ More diverse perspectives  
✅ Handles multi-intent questions  

**Limitations:**
- Higher latency (multiple retrievals)
- More API calls (cost)
- May retrieve redundant info

**When to Use:**
- Broad research questions
- Multi-concept queries
- Comprehensive coverage needed

**Next:** [06_hyde.ipynb](06_hyde.ipynb) - Hypothetical Document Embeddings